In [2]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from get_lat_lon import get_lat_lon_from_address

import tqdm
import requests
from bs4 import BeautifulSoup
import time
import geocoder

In [9]:
# Read recipe inputs
raw_ramen = dataiku.Dataset("raw_ramen_locations")
df = raw_ramen.get_dataframe()

In [10]:
df.head()

,store_id,store_name,address,都,市,区,番地,address_cleaned,score,ward,review_cnt,review,review_star,addresslat,addresslon,addressgranularity


In [4]:
addresses = df['address'].tolist()

In [8]:
def geocoding_lat_lon(address):
    g = geocoder.mapquest(address, key='eOUDWog4FKpjWQmPZWRCzhiKr3GW0mEr')
    latitude = g.json['raw']['latLng']['lat']
    longitude = g.json['raw']['latLng']['lng']
    return latitude, longitude

In [13]:
df['address'].apply(geocoding_lat_lon)

['東京都町田市原町田4-5-18仲見世商店街',
 '東京都渋谷区恵比寿1-1-7１１７ビル\u3000１Ｆ',
 '東京都新宿区山吹町130-15',
 '東京都品川区南大井6-11-10',
 '東京都渋谷区西原3-2-4フロンティア代々木上原 B1F',
 '東京都中野区中野5-56-121F',
 '東京都中野区中野3-36-5',
 '東京都文京区音羽1-17-16中銀音羽マンシオン１F',
 '東京都港区芝1-3-4山谷ビル 1F',
 '東京都豊島区南大塚3-46-5中島ビル 1F',
 '東京都杉並区阿佐谷南3-10-8',
 '東京都青梅市梅郷4-695-1',
 '東京都荒川区東尾久2-19-10',
 '東京都渋谷区桜丘町16-7鈴木ビル 1F',
 '東京都新宿区新宿2-4-1第22宮廷マンション 1F',
 '東京都武蔵野市吉祥寺南町4-16-12リトハイム 101',
 '東京都立川市西砂町1-3-15',
 '東京都港区浜松町1-28-4保木ビル1階',
 '東京都中央区日本橋蛎殻町1-9-5大鷲ビル1F',
 '東京都江東区東陽1-25-6',
 '東京都豊島区巣鴨1-14-1プラトーサカ 1F',
 '東京都板橋区板橋3-5-1リビオタワー板橋 1F',
 '東京都杉並区下高井戸1-22-9',
 '東京都杉並区下井草5-22-17 1Fルメゾン井荻',
 '東京都大田区東馬込1-12-2',
 '東京都府中市本町1-7-2',
 '東京都町田市原町田6-29-4寺田ビル\u3000 １Ｆ',
 '東京都渋谷区神泉町8-13アーガス霜田ビル 1F',
 '東京都品川区平塚2-18-8金泉湯\u3000２Ｆ',
 '東京都中央区八重洲2-1八重洲地下街 外堀地下2番通り',
 '東京都新宿区西新宿7-8-2福八ビル 1F',
 '東京都北区岸町1-1-10岸町NUビル1F',
 '東京都渋谷区恵比寿西1-9-4小熊ビル 1Ｆ',
 '東京都港区南青山2-34-30',
 '東京都立川市錦町1-6-13苅田ビル1F',
 '東京都中野区東中野1-36-7',
 '東京都目黒区下目黒3-2-4',
 '東京都青梅市今寺5-18-49',
 '東京都港区新橋5-9-8永島ビル 1F',
 '東京都小金井市梶野町5-1-1nonowa東小金

In [5]:
lat_lons = get_lat_lon_from_address(addresses)

  1%|          | 5/736 [00:14<34:53,  2.86s/it]


ValueError: Invalid address submitted. 東京都中野区中野5-56-121F

In [11]:
url = 'http://www.geocoding.jp/api/'
latlons = []
address = '東京都中野区中野5-56-121'
payload = {"v": 1.1, 'q': address}
r = requests.get(url, params=payload)
ret = BeautifulSoup(r.content,'lxml')
if ret.find('error'):
    raise ValueError(f"Invalid address submitted. {address}")
else:
    lat = ret.find('lat').string
    lon = ret.find('lng').string
    latlons.append([lat,lon])
    #time.sleep(10)

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

lat_lon_df = raw_ramen_df # For this sample code, simply copy input to output


# Write recipe outputs
lat_lon = dataiku.Dataset("lat_lon")
lat_lon.write_with_schema(lat_lon_df)